<img width="8%" alt="Plotly.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Plotly.png" style="border-radius: 15%">

# Plotly - Create analytics

**Tags:** #plotly #html #csv #image #content #analytics #cohorts

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook creates a chart to follow your growth by cohorts over the weeks.

## Input

### Import libraries

In [ ]:
import plotly.graph_objects as go
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date, datetime, timedelta
import naas_data_product

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Name of the file to be retrieved.
- `linkedin_url`: This variable stores the LinkedIn profile URL that will be used as image source.
- `title`: Graph title
- `col_label`: Label col to be used as x axis
- `col_group`: Group col to be used to be stacked
- `col_value`: Value col to be aggregated on dataframe
- `agg_value`: Type of aggregation to perform on value col

**Outputs**
- `output_dir`: This variable is used for storing the path to the directory where the output files will be saved.
- `csv_output`: CSV output to be saved as picke file.
- `html_output`: HTML output name.
- `image_output`: Image output name to be saved as png file.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
input_file = "sales"
linkedin_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "linkedin_url") or "YOUR_LINKEDIN_URL"
title = "New Sales Qualified Leads"

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
csv_output = os.path.join(output_dir, "sales_trend")
html_output = os.path.join(output_dir, "sales_trend.html")
image_output = os.path.join(output_dir, "sales_trend.png")

## Model

### Get sales qualified leads

In [ ]:
df_init = pload(input_dir, input_file) 
print("⚡️ Sales DB:", len(df_init))
df_init.head(3)

### Create trend dataframe

In [ ]:
def get_trend(
    df_init,
    col_label,
    col_group,
    col_value,
    agg_value,
    rolling=10
):
    # Init
    df = df_init.copy()
    
    # Groupby
    to_group = [
        "ENTITY",
        col_label,
        col_group,
    ]
    to_agg = {
        col_value: agg_value
    }
    to_rename = {
        col_label: "LABEL",
        col_group: "GROUP",
        col_value: "VALUE",
    }
    df = df.groupby(to_group, as_index=False).agg(to_agg).rename(columns=to_rename)
    
    # Filter on this week and last week
    if len(df) > 0:
        tw = date.today().strftime("W%W-%Y")
        lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
        df = df[(df["LABEL"].isin([tw, lw]))]
        colors = {
            tw: "#48DD82",
            lw: "#FFFDA2",
        }
        df["COLOR"] = df["LABEL"].map(colors)

        # Plotly: hovertext
        df["TEXT"] = (
            "<b><span style='font-size: 14px;'>"
            + df["GROUP"].astype(str)
            + ": "
            + df["VALUE"].astype(str)
            + "</span></b>"
        )
    return df.reset_index(drop=True), colors

df_trend, colors = get_trend(
    df_init,
    col_label="SCENARIO",
    col_group="DATE",
    col_value="FULLNAME",
    agg_value="count"
)
df_trend

### Create title and logo

In [ ]:
# Logo
arrow_up = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Eo_circle_green_arrow-up.svg/2048px-Eo_circle_green_arrow-up.svg.png"
arrow_down = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Eo_circle_red_arrow-down.svg/2048px-Eo_circle_red_arrow-down.svg.png"
arrow_right = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Eo_circle_orange_arrow-right.svg/2048px-Eo_circle_orange_arrow-right.svg.png"

# Groupby weeks
df = df_trend.groupby(["LABEL"], as_index=False).agg({"VALUE": "sum"})
total = df.loc[df.index[-1], "VALUE"]
try:
    total_n1 = df.loc[df.index[-2], "VALUE"]
except:
    total_n1 = 0
tw = df.loc[df.index[-1], "LABEL"]
varv = total - total_n1
if total_n1 == 0:
    varp = 1
else:
    varp = varv / total_n1

total_d = "{:,.0f}".format(total).replace(",", " ")
varv_d = "{:,.0f}".format(varv).replace(",", " ")
varp_d = "{:,.0%}".format(varp).replace(",", " ")
if varv >= 0:
    varv_d = f"+{varv_d}"
    varp_d = f"+{varp_d}"
title_full = f"<b><span style='font-size: 20px;'>{title}</span></b><br><span style='font-size: 18px;'>{tw}: {total_d} | {varv_d} ({varp_d}) vs last week</span>"

# Logo
logo = None
if varv > 0:
    logo = arrow_up
elif varv > -0.2:
    logo = arrow_right
else:
    logo = arrow_down

### Create Vertical Barchart stacked

In [ ]:
def get_week_number(week):
    return int(week.split("W")[1].split("-")[0]) 

def create_barchart(
    df,
    label,
    groups,
    value,
    title,
    logo,
):
    # Init
    fig = go.Figure()
    tw_int = get_week_number(tw)
    
    # Create trace
    list_groups = df[groups].unique()
    for group in list_groups:
        showlegend = True
        tmp_df = df[df[groups] == group]
        # Create bar
        fig.add_trace(
            go.Bar(
                x=tmp_df[label],
                y=tmp_df[value],
                name=group,
                marker=dict(color=tmp_df["COLOR"]),
                text=tmp_df[value],
                textposition="outside",
                hoverinfo="text",
                hovertext=tmp_df["TEXT"],
                showlegend=showlegend
            )
        )

    # Add logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom",
        )
    )

    # Display the legend below the graph
    fig.update_layout(legend=dict(
#         orientation="v",
        x=0.25,
        y=-0.2,
#         xref="paper",
#         yref="paper",
        xanchor="left",
        yanchor="bottom",
    ))
    
    
    # Add annotation
    fig.add_annotation(
        text=f"<i>Source: <a href='{linkedin_url}'>{linkedin_url}</a> / Created at: {date.today().isoformat()}</i>",
        font=dict(family="Arial", color="black"),
        x=0,
        y=-0.35,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        arrowcolor="white",
    )
    # Update layout
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        margin_b=120,
        barmode="stack",
        bargap=0.1,  # gap between bars of adjacent location coordinates.
        bargroupgap=0.1,  # gap between bars of the same location coordinate.
    )
    config = {"displayModeBar": False}
    fig.show(config=config)
    return fig

fig = create_barchart(df_trend, label="GROUP", groups="LABEL", value="VALUE", title=title_full, logo=logo)

## Output

### Save data

In [ ]:
pdump(output_dir, df_trend, csv_output)

### Save and share your graph in HTML


In [ ]:
# Save your graph in HTML
fig.write_html(html_output)

# Share output with naas
html_link = naas.asset.add(html_output, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image


In [ ]:
# Save your graph in PNG
fig.write_image(image_output)

# Share output with naas
image_link = naas.asset.add(image_output, override_prod=True)

# -> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)